In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import numpy as np
import polars as pl
import seaborn as sns
import os
import matplotlib.pyplot as plt
import pandas as pd
from tqdm import tqdm
from datetime import datetime

In [ ]:
RAW_DIR = '/content/drive/MyDrive/Colab Notebooks/Nhom_4/2. Thực Hành/Visualize Data/Data'
TRANS_DIR = '/content/drive/MyDrive/Colab Notebooks/Nhom_4/2. Thực Hành/Visualize Data/Translated Data'
PRE_DIR = '/content/drive/MyDrive/Colab Notebooks/Nhom_4/2. Thực Hành/Visualize Data/Preprocessed Data'
WEB_DAT_DIR = '/content/drive/MyDrive/Colab Notebooks/Nhom_4/2. Thực Hành/Visualize Data/WEB DATA'

In [ ]:
def load_valid_users_courses(path):
    df = pl.read_csv(path, separator='\t', has_header=False, new_columns=['user', 'course', 'time'])
    return set(df['user']), set(df['course'])

valid_users, valid_courses = load_valid_users_courses('/content/drive/MyDrive/Colab Notebooks/Nhom_4/2. Thực Hành/Visualize Data/Model Data/interactions_n_core.txt')
len(valid_users), len(valid_courses)

(99969, 2831)

In [ ]:
!pip install googletrans==3.1.0a0

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.1/55.1 kB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.4/133.4 kB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.6/53.6 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.0/65.0 kB 6.2 MB/s eta 0:00:00
  Created wheel for googletrans: filename=googletrans-3.1.0a0-py3-none-any.whl size=16353 sha256=1946cb7b65de6f4db71ba7ae906968c59739bd2e9088d5b36b722a8f3780a94b
  Stored in directory: /root/.cache/pip/wheels/50/5d/3c/8477d0af4ca2b8b1308812c09f1930863caeebc762fe265a95
Successfully built googletrans
  Attempting uninstall: chardet
    Found existing installation: chardet 5.2.0
    Uninstalling cha

In [ ]:
from googletrans import Translator
translator = Translator()

In [ ]:
def translate_str_column(df, col, batch_size=200, chunk_size=10):
    texts = df[col]
    trans_texts = []

    # for i in tqdm(range(0, df.shape[0], batch_size)):
    #     texts_batch = list(texts[i: i + batch_size])
    #     texts_batch = [text.strip() if text else '' for text in texts_batch]
    #     trans_batch = translator.translate(texts_batch, dest='vi')
    #     trans_batch = [trans.text.lower() if trans != '' else None
    #                    for trans in trans_batch]
    #     trans_texts.extend(trans_batch)

    for i in tqdm(range(0, df.shape[0], batch_size)):
        translator = Translator()
        texts_batch = list(texts[i: i + batch_size])
        texts_batch = [text.strip() if text else '' for text in texts_batch]
        texts_chunks = []

        for j in range(0, len(texts_batch), chunk_size):
            texts_chunk = texts_batch[j: j + chunk_size]
            texts_chunk = '\n@@\n'.join(texts_chunk)
            texts_chunks.append(texts_chunk)

        trans_chunks = translator.translate(texts_batch, dest='vi')
        trans_chunks = [trans_texts for trans_chunk in trans_chunks for trans_texts in trans_chunk.text.split('\n@@\n')]
        trans_texts.extend(trans_chunks)
        assert len(trans_chunks) == len(texts_batch)
    assert len(trans_texts) == df.shape[0]

    df = df.with_columns(pl.Series(name=f"{col}_trans", values=trans_texts))

    return df

def translate_cate_column(df, col, batch_size=200):
    # Strip texts
    df = df.with_columns(pl.col(col).str.strip())

    # Get all categories
    categories = set(df[col])

    if None in categories:
        categories.remove(None)

    if '' in categories:
        categories.remove('')

    categories = list(categories)

    # Dịch thể loại và tạo mapping
    mapping = {None: None, '': None}

    for i in tqdm(range(0, len(categories), batch_size)):
        batch_cates = categories[i : i + batch_size]
        trans_cates = translator.translate(batch_cates, dest='vi')
        trans_cates = [trans.text.lower() for trans in trans_cates]
        assert len(batch_cates) == len(trans_cates)

        for j in range(len(batch_cates)):
            mapping[batch_cates[j]] = trans_cates[j]

    # Ánh xạ tiếng Trung qua tiếng Việt
    df = df.with_columns(pl.col(col).map_elements(lambda x: mapping[x]))

    return df

def translate_cate_list_column(df, col, batch_size=500):
    # Strip texts
    df = df.with_columns(pl.col(col).list.eval(pl.element().str.strip()))

    # Lưu các thể loại
    categories = {cate for row_cates in df[col] for cate in row_cates}

    if None in categories:
        categories.remove(None)

    if '' in categories:
        categories.remove('')

    categories = list(categories)

    # Dịch thể loại và tạo mapping
    mapping = {None: None, '': ''}

    for i in tqdm(range(0, len(categories), batch_size)):
        batch_cates = categories[i : i + batch_size]
        trans_cates = translator.translate(batch_cates, dest='vi')
        trans_cates = [trans.text.lower() for trans in trans_cates]
        assert len(batch_cates) == len(trans_cates)

        for j in range(len(batch_cates)):
            mapping[batch_cates[j]] = trans_cates[j]

    # Ánh xạ tiếng Trung qua tiếng Việt
    df = df.with_columns(pl.col(col).map_elements(lambda row_cates: [mapping[cate] for cate in row_cates]))

    return df

## 1. Users, Courses, Fields, Videos, Exercises

In [ ]:
course_df = pl.read_ndjson(os.path.join(PRE_DIR, 'entities/course_eda.json'))
course_df

id,name,prerequisites,about,resource,name_trans,about_trans,about_segmented,name_segmented,field
str,str,str,str,list[struct[3]],str,str,str,str,list[str]
"""C_584313""","""《资治通鉴》导读""","""""","""通过老师导读，同学们可深入这…","[{[""第一课 导论与三家分晋"", ""导论"", ""导论""],""V_849"",""1.1.1""}, {[""第一课 导论与三家分晋"", ""智伯的覆亡"", ""智伯的覆亡""],""V_850"",""1.2.1""}, … {[""第十五课 隋唐霸业"", null, ""第十五课 隋唐霸业--习题""],""Ex_957"",""15.8""}]","""giới thiệu về …","""thông qua sự h…","""thông_qua sự h…","""giới_thiệu về …","[""lịch sử"", ""trường dạy tiếng trung""]"
"""C_584329""","""微积分——极限理论与一元函数…","""""","""本课程是理工科的一门数学基础…","[{[""序言"", ""序言"", ""序言""],""V_1350"",""1.1.1""}, {[""第一章 实数与函数"", ""第一节 实数集的界与确界"", ""实数集的界""],""V_1351"",""2.1.1""}, … {[""第八章 级数"", null, ""第八章 级数--第六节思考与练习""],""Ex_1545"",""9.9""}]","""giải tích - lý…","""khóa học này l…","""khoá học này l…","""giải_tích - lý…","[""toán học"", ""kinh tế học ứng dụng"", … ""kinh tế lý thuyết""]"
"""C_584381""","""新闻摄影""","""""","""掌握基本的摄影技能，了解图片…","[{[""第一章 绪论"", ""第一讲 引言1"", ""引言1""],""V_1800"",""1.1.1""}, {[""第一章 绪论"", ""第二讲 引言2"", ""引言2""],""V_1801"",""1.2.1""}, … {[""大作业提交"", null, ""《大作业》提交--小节""],""Ex_1926"",""20.4""}]","""chụp ảnh tin t…","""nắm vững các k…","""nắm vững các k…","""chụp ảnh tin_t…","[""báo chí"", ""nghệ thuật""]"
"""C_597208""","""数据挖掘：理论与算法""","""""","""最有趣的理论+最有用的算法=…","[{[""走进数据科学：博大精深，美不胜收"", ""整装待发"", ""Video""],""V_2961"",""1.1.1""}, {[""走进数据科学：博大精深，美不胜收"", ""学而不思则罔"", ""Video""],""V_2962"",""1.3.1""}, … {[""美丽数据说：阆苑仙葩，美玉无瑕"", null, ""第十一章第一节测试题""],""Ex_3104"",""11.1""}]","""khai thác dữ l…","""lý thuyết thú …","""lý_thuyết thú_…","""khai_thác dữ_l…","[""khoa học và công nghệ máy tính""]"
"""C_597225""","""大学计算机""","""""","""大学计算机课程将以计算思维为…","[{[""第1周： 基于计算机的问题求解"", ""课程介绍"", ""开篇""],""V_4596"",""1.1.1""}, {[""第1周： 基于计算机的问题求解"", ""1.0 本章导学"", ""1.0 本章导学""],""V_4597"",""1.2.1""}, … {[""第9周：算法与程序设计"", null, ""第九周测验""],""Ex_4827"",""10.12""}]","""máy tính đại h…","""các khóa học m…","""các khoá học m…","""máy_tính đại_h…",null
"""C_597229""","""财务分析与决策""","""""","""这门课程用财务语言解构企业的…","[{[""资金的运用——认识资产"", ""1.1 绪论"", ""绪论""],""V_5042"",""2.1.1""}, {[""资金的运用——认识资产"", ""1.2 认识资产负债表"", ""认识资产负债表""],""V_5043"",""2.2.1""}, … {[""期末大作业——主观题"", null, ""期末主观题""],""Ex_5144"",""12.2""}]","""phân tích tài …","""khóa học này s…","""khoá học này s…","""phân_tích tài_…","[""kinh tế học ứng dụng"", ""khoa học và kỹ thuật quản lý""]"
"""C_597291""","""高级英语写作""","""""","""本课程能够帮助学生掌握英语段…","[{[""Chapter One Paragraph Writing"", ""1.1 Parts of a Paragraph"", ""1.1 Parts of a Paragraph""],""V_8379"",""1.1.1""}, {[""Chapter One Paragraph Writing"", ""1.2 Four Steps in Writing"", ""1.2.1 Step 1- Begin with a point""],""V_8380"",""1.2.1""}, … {[""Chapter Five Research Paper Writing"", null, ""Chapter Five Research Paper Writing""],""Ex_8432"",""5.3""}]","""viết tiếng anh…","""khóa học này c…","""khoá học này c…","""viết tiếng anh…",null
"""C_597307""","""大唐兴衰""","""""","""隋唐五代史是史学名著《资治通…","[{[""第一课、隋朝开基"", ""第一节 隋帝杨坚"", ""第一节 隋帝杨坚""],""V_9394"",""1.1.1""}, {[""第一课、隋朝开基"", ""第二节 杨隋代周"", ""第二节 杨隋代周""],""V_9395"",""1.2.1""}, … {[""第十二课、落日长安"", null, ""第十二课、落日长安--习题""],""Ex_9464"",""12.5""}]","""sự thăng trầm …","""lịch sử nhà tù…","""lịch_sử nhà tu…","""sự thăng_trầm …","[""lịch sử""]"
"""C_597365""","""五分钟轻松搞定职场礼仪（20…","""""","""职场“礼”为先，成功的未来不…","[{[""课程介绍动画：职场“礼”为先，成功的未来不是梦"", ""课程介绍动画"", ""课程介绍动画 — 职场“礼”为先，成功的未来不是梦""],""V_15713"",""1.1.1""}, {[""第一章 塑造优雅、大方、可亲的职业形象"", ""第一节 坐姿篇"", ""本讲导学 坐姿""],""V_15714"",""2.1.1""}, … {[""第五章 应对繁杂、多变、细致的文化差异"", ""第四节 欧美篇"", ""授课视频 讲义 — 欧美各国礼仪""],""V_15800"",""6.4.1""}]","""dễ dàng nắm vữ…","""“nghi thức” đư…","""“ nghi_thức ” …","""dễ_dàng nắm vữ…",null


In [ ]:
course_df = course_df.filter(pl.col('id').is_in(valid_courses))
course_df.shape

(2831, 10)

In [ ]:
course_tab = course_df.select('id', 'name_trans', 'about_trans', 'prerequisites') \
                        .rename({'name_trans': 'name', 'about_trans': 'about'}) \
                        .with_columns(
                            pl.when(pl.col(pl.Utf8).exclude('id').str.lengths() == 0) \
                            .then(None).otherwise(pl.col(pl.Utf8).exclude('id')).keep_name()
                        )
course_tab

In [ ]:
course_tab = translate_str_column(course_tab, 'prerequisites', 200)
course_tab

100%|██████████| 15/15 [01:06<00:00,  4.46s/it]


id,name,about,prerequisites,prerequisites_trans
str,str,str,str,str
"""C_655852""","""chuỗi khóa học…","""hiện nay, khoả…",null,""""""
"""C_655850""","""chuỗi lớp học …","""hiện nay, khoả…",null,""""""
"""C_654554""","""chuỗi khóa học…","""hiện nay, khoả…",null,""""""
"""C_654506""","""chuỗi khóa học…","""hiện nay, khoả…",null,""""""
"""C_629558""","""miễn dịch học …","""hiện nay, khoả…",null,""""""
"""C_629503""","""kiến thức thôn…","""hiện nay, khoả…",null,""""""
"""C_597314""","""cao đẳng văn h…","""hiện nay, khoả…",null,""""""
"""C_674903""","""nghệ thuật bất…","""[khóa học chất…","""无""","""không có"""
"""C_674910""","""tính thẩm mỹ v…","""[khóa học chất…",null,""""""


In [ ]:
course_tab.drop('prerequisites').rename({'prerequisites_trans': 'prerequisites'}).write_csv(os.path.join(WEB_DAT_DIR, 'courses.csv'))

## fields, course-fields

In [ ]:
course_fields = course_df.select(['id', 'field']).explode('field').drop_nulls()
course_fields

id,field
str,str
"""C_629503""","""quản lý thông …"
"""C_597314""","""trường dạy tiế…"
"""C_674903""","""nghệ thuật"""
"""C_674920""","""triết lý"""
"""C_674950""","""địa lý"""
"""C_674962""","""trường dạy tiế…"
"""C_674968""","""dân tộc học"""
"""C_674968""","""trường dạy tiế…"
"""C_676642""","""quản lý kinh d…"


In [ ]:
fields = course_fields['field'].unique().to_list()
field_tab = pl.DataFrame({
    'id': range(len(fields)),
    'name': fields
})
f_map = dict(zip(field_tab['name'], field_tab['id']))

In [ ]:
course_fields = course_fields.with_columns(
        pl.col('field').map_dict(f_map).cast(pl.Int64)
    ).rename({'id': 'course_id', 'field': 'field_id'})

course_fields

In [ ]:
field_tab.write_csv(os.path.join(WEB_DAT_DIR, 'fields.csv'))
course_fields.write_csv(os.path.join(WEB_DAT_DIR, 'course_fields.csv'))

## videos, exercises

In [ ]:
vid_dict = dict()
ex_dict = dict()
for row in tqdm(course_df.select(['id', 'resource']).iter_rows()):
    course_id, resrcs = row
    for resrc in resrcs:
        titles, resrc_id, chapter = resrc['titles'], resrc['resource_id'], resrc['chapter']

        if resrc_id[0] == 'V':
            resrc_dict = vid_dict
        else:
            resrc_dict = ex_dict

        if resrc_id not in resrc_dict:
            resrc_dict[resrc_id] = []
        resrc_dict[resrc_id].append((titles, chapter))

2831it [00:03, 790.20it/s]


In [ ]:
for vid_id, val in ex_dict.items():
    if (len(val) > 1):
        print(val)

Như vậy, mỗi video, mỗi bài tập sẽ chỉ tương ứng với một khóa học

In [ ]:
vid_dict = {"id": [], "titles": [], 'chapter': [], 'course_id': []}
ex_dict = {"id": [], "titles": [], 'chapter': [], 'course_id': []}

for row in tqdm(course_df.select(['id', 'resource']).iter_rows()):
    course_id, resrcs = row
    for resrc in resrcs:
        titles, resrc_id, chapter = resrc['titles'], resrc['resource_id'], resrc['chapter']

        if titles:
            titles = [title.strip() if title else '' for title in titles]
            titles = '@'.join(titles)
        else:
            titles = None

        if chapter.strip() == '':
            chapter = None

        if not resrc_id:
            continue

        if resrc_id[0] == 'V':
            resrc_dict = vid_dict
        else:
            resrc_dict = ex_dict

        resrc_dict['id'].append(resrc_id)
        resrc_dict['titles'].append(titles)
        resrc_dict['chapter'].append(chapter)
        resrc_dict['course_id'].append(course_id)

vid_tab = pl.from_dict(vid_dict)
ex_tab = pl.from_dict(ex_dict)

2831it [00:02, 1040.43it/s]


In [ ]:
vid_tab

id,titles,chapter,course_id
str,str,str,str
"""V_32148""","""第一周 绪论@开场白@开场白…","""1.1.1""","""C_655852"""
"""V_32151""","""第一周 绪论@绪论@绪论""","""1.2.1""","""C_655852"""
"""V_32154""","""第一周1中医理论-阴阳五行@…","""2.1.1""","""C_655852"""
"""V_32156""","""第一周1中医理论-阴阳五行@…","""2.3.1""","""C_655852"""
"""V_32157""","""第一周1中医理论-阴阳五行@…","""2.3.2""","""C_655852"""
"""V_32160""","""第一周1中医理论-阴阳五行@…","""2.4.1""","""C_655852"""
"""V_32164""","""第二周2中医理论-五脏六腑@…","""3.1.1""","""C_655852"""
"""V_32166""","""第二周2中医理论-五脏六腑@…","""3.3.1""","""C_655852"""
"""V_32168""","""第二周2中医理论-五脏六腑@…","""3.4.1""","""C_655852"""


In [ ]:
ex_tab

id,titles,chapter,course_id
str,str,str,str
"""Ex_32152""","""第一周 绪论@@绪论课堂小测…","""1.3""","""C_655852"""
"""Ex_32153""","""第一周 绪论@@绪论--绪论…","""1.4""","""C_655852"""
"""Ex_32161""","""第一周1中医理论-阴阳五行@…","""2.2""","""C_655852"""
"""Ex_32162""","""第一周1中医理论-阴阳五行@…","""2.5""","""C_655852"""
"""Ex_32163""","""第一周1中医理论-阴阳五行@…","""2.6""","""C_655852"""
"""Ex_32179""","""第二周2中医理论-五脏六腑@…","""3.2""","""C_655852"""
"""Ex_32180""","""第二周2中医理论-五脏六腑@…","""3.5""","""C_655852"""
"""Ex_32181""","""第二周2中医理论-五脏六腑@…","""3.8""","""C_655852"""
"""Ex_32182""","""第二周2中医理论-五脏六腑@…","""3.11""","""C_655852"""


In [ ]:
vid_tab.write_csv(os.path.join(WEB_DAT_DIR, 'videos.csv'))
ex_tab.write_csv(os.path.join(WEB_DAT_DIR, 'exercises.csv'))

# 2. Videos, Exercises, Course_Videos, Course_Exercises

In [ ]:
user_df = pl.read_ndjson(os.path.join(PRE_DIR, 'entities/user_eda.json'))
user_df

id,name,gender,school,course_order,enroll_time
str,str,i64,str,list[i64],list[str]
"""U_22""","""我""",0,null,[682129],"[""2019-10-12 10:28:02""]"
"""U_24""","""王帅国""",1,"""清华大学""","[597314, 597208, … 2229905]","[""2019-06-12 17:22:07"", ""2019-06-17 15:22:41"", … ""2020-11-21 11:38:57""]"
"""U_25""","""王帅国""",0,"""清华大学""",[1903985],"[""2020-08-07 18:59:13""]"
"""U_53""","""于歆杰""",1,"""清华大学""","[696679, 943255, … 1794464]","[""2020-03-01 21:24:30"", ""2020-03-17 08:46:12"", … ""2020-06-18 18:47:15""]"
"""U_54""","""马昱春""",2,"""清华大学""","[682442, 682164, … 1793742]","[""2019-10-09 02:17:49"", ""2019-11-08 00:49:03"", … ""2020-06-13 15:29:18""]"
"""U_67""","""李小马""",1,"""学堂在线""",[696679],"[""2019-12-20 12:06:00""]"
"""U_68""","""秋""",2,"""清华大学""","[696692, 948431]","[""2020-01-21 10:15:02"", ""2020-01-21 10:18:02""]"
"""U_69""","""培源_""",1,"""清华大学""","[584329, 584313, … 875624]","[""2019-05-08 15:15:56"", ""2019-05-08 15:33:30"", … ""2019-11-05 11:51:59""]"
"""U_90""","""刘俊洋""",0,"""qinghua""","[676664, 707135, … 1824921]","[""2020-04-22 18:23:28"", ""2020-04-22 21:12:21"", … ""2020-06-22 15:47:55""]"


In [ ]:
user_df = user_df.filter(pl.col('id').is_in(valid_users)).select(['id', 'name', 'gender', 'school'])
user_df

id,name,gender,school
str,str,i64,str
"""U_146""","""张幸福""",1,null
"""U_185""","""教师-陈燕秀""",2,"""贵州理工学院"""
"""U_205""","""尹亮""",1,"""青海大学"""
"""U_217""","""饲猫少年""",0,"""加利盾分校"""
"""U_464""","""教师-安宇""",1,"""清华大学"""
"""U_835""","""汪潇潇""",0,"""清华大学"""
"""U_981""","""杨芳""",2,null
"""U_1064""","""王大亮""",2,null
"""U_1195""","""胡欣宇""",0,"""山西农业大学"""


In [ ]:
old_user_df = pl.read_ndjson(os.path.join(RAW_DIR, 'entities/user.json')) \
                .select(['id', 'year_of_birth']) \
                .filter(pl.col('id').is_in(valid_users))
old_user_df

id,year_of_birth
str,str
"""U_146""",null
"""U_185""",null
"""U_205""",null
"""U_217""",null
"""U_464""",null
"""U_835""",null
"""U_981""",null
"""U_1064""",null
"""U_1195""",null


In [ ]:
old_user_df['year_of_birth'].is_null().sum()

99969

In [ ]:
user_df = user_df.with_columns(
        pl.when(pl.col('name').str.lengths() != 0).then(pl.col('name')),
        pl.when(pl.col('school').str.lengths() != 0).then(pl.col('school'))
    ).join(old_user_df, on='id', how='left')

user_df

<ipython-input-148-74d87c4170f6>:2: DeprecationWarning: `lengths` is deprecated. It has been renamed to `len_bytes`.
  pl.when(pl.col('name').str.lengths() != 0).then(pl.col('name')),
<ipython-input-148-74d87c4170f6>:3: DeprecationWarning: `lengths` is deprecated. It has been renamed to `len_bytes`.
  pl.when(pl.col('school').str.lengths() != 0).then(pl.col('school'))


id,name,gender,school,year_of_birth
str,str,i64,str,str
"""U_146""","""张幸福""",1,null,null
"""U_185""","""教师-陈燕秀""",2,"""贵州理工学院""",null
"""U_205""","""尹亮""",1,"""青海大学""",null
"""U_217""","""饲猫少年""",0,"""加利盾分校""",null
"""U_464""","""教师-安宇""",1,"""清华大学""",null
"""U_835""","""汪潇潇""",0,"""清华大学""",null
"""U_981""","""杨芳""",2,null,null
"""U_1064""","""王大亮""",2,null,null
"""U_1195""","""胡欣宇""",0,"""山西农业大学""",null


In [ ]:
# user_df = translate_cate_column(user_df, 'school')
# user_df = translate_cate_column(user_df, 'name')
# user_df.write_csv(os.path.join(WEB_DAT_DIR, 'users.csv'))

<ipython-input-143-1d207b22024d>:21: DeprecationWarning: `strip` is deprecated. It has been renamed to `strip_chars`.
  df = df.with_columns(pl.col(col).str.strip())
100%|██████████| 15/15 [04:24<00:00, 17.63s/it]
<ipython-input-143-1d207b22024d>:47: PolarsInefficientMapWarning: 
Expr.map_elements is significantly slower than the native expressions API.
Only use if you absolutely CANNOT implement your logic otherwise.
Replace this expression...
  - pl.col("school").map_elements(lambda x: ...)
with this one instead:
  + pl.col("school").replace(mapping)

  df = df.with_columns(pl.col(col).map_elements(lambda x: mapping[x]))
<ipython-input-143-1d207b22024d>:21: DeprecationWarning: `strip` is deprecated. It has been renamed to `strip_chars`.
  df = df.with_columns(pl.col(col).str.strip())
 41%|████▏     | 131/317 [23:16<33:03, 10.66s/it]


KeyboardInterrupt: 

In [ ]:
user_courses = pl.read_csv('/content/drive/MyDrive/Colab Notebooks/Nhom_4/2. Thực Hành/Visualize Data/Model Data/interactions_n_core.txt',
                 separator='\t', has_header=False, new_columns=['user', 'course', 'time']) \
                 .rename({'user': 'user_id', 'course': 'course_id', 'time': 'enroll_time'})
user_courses.write_csv(os.path.join(WEB_DAT_DIR, 'user_courses.csv'))
user_courses

user_id,course_id,enroll_time
str,str,str
"""U_146""","""C_680824""","""2020-02-03 22:…"
"""U_146""","""C_766203""","""2020-02-03 22:…"
"""U_146""","""C_873864""","""2020-02-03 22:…"
"""U_146""","""C_682443""","""2020-02-19 20:…"
"""U_146""","""C_682222""","""2020-02-19 21:…"
"""U_146""","""C_680845""","""2020-04-14 20:…"
"""U_185""","""C_696729""","""2019-09-18 10:…"
"""U_185""","""C_707096""","""2019-11-27 08:…"
"""U_185""","""C_943255""","""2019-12-28 16:…"


In [ ]:
def generate_name_by_gender(g):
    if g == 0:
        g = 1
    elif g == 1:
        g =  0
    else:
        g = random.randint(0, 1)

    return generator.generate(g)

user_df = pl.read_csv(os.path.join('data/users_need_trans.csv'))
user_df = user_df.with_columns(
    pl.col('gender').map_elements(
        lambda g: generate_name_by_gender(g)
    ).alias('name')
)

user_df.write_csv('data/users_namegen.csv')

# 3. Concepts, Teachers, Schools (Local)

In [ ]:
import json
import polars as pl
import os
from vn_fullname_generator import generator
import random

def load_valid_users_courses(path):
    df = pl.read_csv(path, separator='\t', has_header=False, new_columns=['user', 'course', 'time'])
    return set(df['user']), set(df['course'])

def prepare_users(path, valid_users):
    df = pl.read_ndjson(path)
    df = df.filter(pl.col('id').is_in(valid_users))
    print(len(df))

def prepare_course_teachers():
    valid_courses = set(pl.read_csv('raw-data/interactions_n_core.txt', separator='\t', has_header=False, new_columns=['user', 'course', 'time'])['course'])
    teacher_df = pl.read_ndjson('raw-data/teacher_eda.json') \
                    .select(pl.all().exclude('about')) \
                    .rename({'about_trans': 'about'}) \
                    .with_columns(
                        pl.when(pl.all().exclude('id').str.strip().str.lengths() != 0).then(pl.all().exclude('id').str.strip())
                    )
    course_teacher_df = pl.read_csv('raw-data/course-teacher_eda.txt', separator='\t', has_header=False, new_columns=['course_id', 'teacher_id'])

    # Filter invalid course
    old_shape = course_teacher_df.shape
    course_teacher_df = course_teacher_df.filter(pl.col('course_id').is_in(valid_courses)).unique()
    new_shape = course_teacher_df.shape
    print(f'Filtering association ==> Before: {old_shape}; New: {new_shape}')
    course_teacher_df.write_csv('data/course_teachers.csv')

    # Add new teachers
    old_shape = teacher_df.shape
    asso_teachers = set(course_teacher_df['teacher_id'])
    new_teachers = asso_teachers.difference(set(teacher_df['id']))
    print('# new teachers: ', len(new_teachers))

    if new_teachers:
        new_teacher_df = pl.from_dict({
            'id': list(new_teachers),
        })

        teacher_df = pl.concat([teacher_df, new_teacher_df], how='diagonal')

    teacher_df = teacher_df.unique(subset=['id'])
    teacher_df.write_csv('data/teachers.csv')

def prepare_course_schools():
    valid_courses = set(pl.read_csv('raw-data/interactions_n_core.txt', separator='\t', has_header=False, new_columns=['user', 'course', 'time'])['course'])

    # school_df = pl.read_ndjson('raw-data/school.json')
    # print(school_df.columns)

    school_df = pl.read_ndjson('raw-data/school.json') \
                    .with_columns(
                        pl.when(pl.all().exclude('id').str.strip().str.lengths() != 0).then(pl.all().exclude('id').str.strip())
                    )

    course_school_df = pl.read_csv('raw-data/course-school_eda.txt', separator='\t', has_header=False, new_columns=['course_id', 'school_id'])

    # Filter invalid course
    old_shape = course_school_df.shape
    course_school_df = course_school_df.filter(pl.col('course_id').is_in(valid_courses)).unique()
    new_shape = course_school_df.shape
    print(f'- Filtering association ==> Before: {old_shape}; New: {new_shape}')
    course_school_df.write_csv('data/course_schools.csv')

    # Add new teachers
    old_shape = school_df.shape
    asso_schools = set(course_school_df['school_id'])
    new_schools = asso_schools.difference(set(school_df['id']))
    print('- # new schools: ', len(new_schools))

    if new_schools:
        new_school_df = pl.from_dict({
            'id': list(new_schools),
        })

        school_df = pl.concat([school_df, new_school_df], how='diagonal')

    school_df = school_df.unique(subset=['id'])
    school_df.write_csv('data/schools.csv')

def prepare_course_concepts():
    valid_courses = set(pl.read_csv('raw-data/interactions_n_core.txt', separator='\t', has_header=False, new_columns=['user', 'course', 'time'])['course'])

    concept_df = pl.read_ndjson('raw-data/concept.json') \
                    .select(['id', 'name']) \
                    .with_columns(
                        pl.when(pl.all().exclude('id').str.strip().str.lengths() != 0).then(pl.all().exclude('id').str.strip())
                    )

    course_concept_df = pl.read_csv('raw-data/concept-course_eda.txt', separator='\t', has_header=False, new_columns=['concept_id', 'course_id']) \
                            .select(['course_id', 'concept_id'])

    # Filter invalid course
    # Do có 1 số concept ids như nhau chỉ khác là viết hoa viết thường --> upper case rồi unique
    # Duplicate: K_HEMIPTERA; K_共射(电路）_电子科学与技术
    # K_共射（电路）_电子科学与技
    old_shape = course_concept_df.shape
    course_concept_df = course_concept_df.filter(pl.col('course_id').is_in(valid_courses)) \
                                        .with_columns(pl.col('concept_id').str.to_uppercase()) \
                                        .unique()

    new_shape = course_concept_df.shape
    print(f'- Filtering association ==> Before: {old_shape}; New: {new_shape}')
    course_concept_df.write_csv('data/course_concepts.csv')

    # Add new concepts
    old_shape = concept_df.shape
    concept_df = concept_df.with_columns(pl.col('id').str.to_uppercase())
    asso_concepts = set(course_concept_df['concept_id'])
    new_concepts = asso_concepts.difference(set(concept_df['id']))
    print('- # new concepts: ', len(new_concepts))

    if new_concepts:
        new_concept_df = pl.from_dict({
            'id': list(new_concepts),
        })

        concept_df = pl.concat([concept_df, new_concept_df], how='diagonal')

    concept_df = concept_df.unique(subset=['id'])
    concept_df.write_csv('data/concepts.csv')

In [ ]:
PRE_DIR = '../preprocessed-data'

valid_users, valid_courses = load_valid_users_courses(os.path.join(PRE_DIR, 'interactions_n_core.txt'))

prepare_users(os.path.join(PRE_DIR, 'entities/user_eda.json'), valid_users)
prepare_course_teachers()
prepare_course_schools()
prepare_course_concepts()

# 4. Create sql files

In [ ]:
def create_sql_file(in_path, out_path):
    df = pl.read_csv(in_path, infer_schema_length=10000)
    wf = open(out_path, 'w', encoding='utf-8')

    wf.write('use mooccubex;\n\n')

    tab_name = os.path.basename(in_path).split('.')[0]
    wf.write(f'insert into {tab_name}\nvalues\n')

    for i, row in enumerate(df.iter_rows()):
        wf.write('(')
        for idx, ele in enumerate(row):
            if ele is None:
                ele = 'null'
            else:
                ele = ele.replace('"', '') if isinstance(ele, str) else ele
                ele = f'"{ele}"'

            if idx == 0:
                wf.write(f'{ele}')
            else:
                wf.write(f',{ele}')

        if i != df.shape[0] - 1:
            wf.write('),\n')
        else:
            wf.write(');')

    wf.close()

def combine_sql_files(in_paths, out_path):
    wf = open(out_path, 'w', encoding='utf-8')

    # Use these line to increase connection time to inser large data
    wf.write('''SET GLOBAL max_allowed_packet=1073741824;
SET GLOBAL wait_timeout = 600;
SET GLOBAL net_read_timeout = 600;
SET GLOBAL connect_timeout = 600;
''')

    for in_path in in_paths:
        rf = open(in_path, 'r', encoding='utf-8')
        for line in rf.readlines():
            wf.write(line)

        wf.write('\n\n\n')

        rf.close()

    wf.close()

In [ ]:
out_folder = 'sql-files/insert'

for fname in os.listdir('data'):
    input_path = os.path.join('data', fname)
    output_path = os.path.join(out_folder, fname.split('.')[0] + '.sql')
    create_sql_file(input_path, output_path)

In [ ]:
in_paths = [
        'courses.sql', 'users.sql', 'videos.sql', 'exercises.sql',
        'teachers.sql', 'schools.sql', 'fields.sql', 'concepts.sql',
        'user_courses.sql', 'course_concepts.sql', 'course_fields.sql',
        'course_teachers.sql', 'course_schools.sql'
    ]

in_paths = ['sql-files/insert/' + path for path in in_paths]

combine_sql_files(in_paths, 'sql-files/insert_data.sql')

In [ ]:
def check_length(folder):
    for fname in os.listdir(folder):
        print(f'===== Process {fname} =====')
        path = os.path.join(folder, fname)
        if fname in ('exercises.csv', 'videos.csv'):
            df = df = pl.read_csv(path, schema={
                'id': pl.Utf8,
                'titles': pl.Utf8,
                'chapter': pl.Utf8,
                'course_id': pl.Utf8,
            })
        else:
            df = pl.read_csv(path)

        df = df.with_columns(pl.col(pl.Utf8).str.lengths()).max()
        print(df)

check_length(folder_data)